# Causal Analysis of Symmetric Facts
We analyze where facts might be locateed in an LLM. A fact is represented by a tuple of subject, relation, object (s,r,o). We also investigate where an inverse fact mis located. An inverse fact is represented by the tuple (o, r^-1, s). For example, the fact "Paris is the capital of France" has the inverse fact "France's capital is France". 

## Setup

In [1]:
try:
    import google.colab
    IN_COLAB = True
    # %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
    renderer = "colab"
except:
    IN_COLAB = False
    from IPython import get_ipython
    %load_ext autoreload
    %autoreload 2
    renderer = "jupyterlab"

In [2]:
%%bash
cd ../
pip install poetry
poetry install
cd notebooks

  Obtaining dependency information for platformdirs<4.0.0,>=3.0.0 from https://files.pythonhosted.org/packages/14/51/fe5a0d6ea589f0d4a1b97824fb518962ad48b27cd346dcdfa2405187997a/platformdirs-3.10.0-py3-none-any.whl.metadata
  Using cached platformdirs-3.10.0-py3-none-any.whl.metadata (11 kB)
Using cached platformdirs-3.10.0-py3-none-any.whl (17 kB)
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.8.0
    Uninstalling platformdirs-3.8.0:
      Successfully uninstalled platformdirs-3.8.0
Installing dependencies from lock file

Package operations: 0 installs, 1 update, 0 removals

  • Downgrading platformdirs (3.10.0 -> 3.8.0)

Installing the current project: transformer-lens (0.0.0)


In [3]:

# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio
pio.renderers.default = renderer

In [4]:


# Import stuff
import types
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader
import json
from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy
import subprocess

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
from tqdm import tqdm

In [5]:
import ast
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib.cm import ScalarMappable
import pickle

In [6]:
if IN_COLAB: 
    import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens.utilities import devices
from transformer_lens.past_key_value_caching import HookedTransformerKeyValueCache
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [7]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [8]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def cuda():
    return torch.cuda.is_available()

def get_device(): 
    return "cuda" if cuda() else "cpu"

def save_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

# Function to load a pickle object from a file
def load_pickle(filename):
    with open(filename, 'rb') as f:
        obj = pickle.load(f)
    return obj

def save_json(data, file_path):
    with open(file_path, "w") as json_file:
        json.dump(data, json_file, indent=4)

def load_json(file_path):
    with open(file_path, "r") as json_file:
        loaded_data = json.load(json_file)
    return loaded_data

def get_gpu_usage(): 
    subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

device = get_device()
device

'cuda'

In [9]:
device = "cpu"

## Model

- fold_ln: Whether to fold in the LayerNorm weights to the subsequent linear layer. This does not change the computation.
- center_writing_weights: Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNormthis doesn't change the computation.
- center_unembed : Whether to center W_U (ie set mean to be zero). Softmax is translation invariant so this doesn't affect log probs or loss, but does change logits. Defaults to True.
- refactor_factored_attn_matrices: Whether to convert the factoredmatrices (W_Q & W_K, and W_O & W_V) to be "even". 

In [10]:
MODEL_NAME = "gpt2-large"
model = HookedTransformer.from_pretrained(
        MODEL_NAME,
        center_unembed=True,  
        center_writing_weights=True,              # Whether to center weights writing to the residual stream (ie set mean to be zero). Due to LayerNorm this doesn't change the computation.      
        fold_ln=True,                             # Whether to  fold in the LayerNorm weights to the subsequent linear layer.
        refactor_factored_attn_matrices=True,
    )


 

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-large into HookedTransformer


## Dataset
We use the CounterFact Dataset

In [11]:

dataset = load_json("data/fact_dataset.json")

In [12]:
def sample_dataset(dataset, idx = None): 
    if idx is None: 
        prompt = "The {} is located in"
        subject = "Eiffel Tower"
        target = "Paris"
    else: 
        sample = dataset[idx]
        prompt = sample["requested_rewrite"]["prompt"]
        subject = sample["requested_rewrite"]["subject"]
        target = sample["requested_rewrite"]["target_true"]["str"]
    return prompt, subject, target

In [13]:
sample = sample_dataset(dataset)
prompt, subject, target = sample
prompt = prompt.format(subject)

In [14]:
model.to_tokens(prompt).shape

torch.Size([1, 9])

## Ablation Methods: 
- Noise ablation 
- Resample ablation

In [15]:
def noise_ablation(prompt, subject, target, n_noise_samples=5, vx=3):
    subject_tokens = model.to_tokens(subject)
    
    #shape: batch, n_tokens, embedding_dim
    subject_embedding = model.embed(subject_tokens)
    _, n_tokens, embedding_dim = subject_embedding.shape
    
    #noise: N(0,v), v = 3*std(embedding)
    embedding = model.W_E
    v = vx*torch.std(embedding, dim=0) #for each v in V
    noise = torch.randn(
        (n_noise_samples, n_tokens, embedding_dim)
    ).to(device) + v
    
    subject_embedding_w_noise = subject_embedding + noise
    
    #shape: batch, n_tokens, vocab_size (logits)
    unembedded_subject = model.unembed(subject_embedding_w_noise)

    noisy_subject_tokens = torch.argmax(unembedded_subject, dim=-1)
    noisy_subject_str = [
        model.to_string(nst) for nst in noisy_subject_tokens
    ]
    true_prompt = prompt.format(subject)
    corrupted_prompts = [
        prompt.format(nss) for nss in noisy_subject_str
    ]
    return true_prompt, corrupted_prompts, target

In [16]:
prompt, subject, target = sample_dataset(dataset)
noise_ablation(prompt, subject, target)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [17]:
def resample_ablation(prompt, subject, target, n_noise_samples=20):
    subject_tokens = model.to_tokens(subject)
    embedding = model.W_E
    #we select n random rows from the embedding matrix
    permutations = torch.randperm(embedding.size(0))[:n_noise_samples]
    random_samples = embedding[permutations]
    #unsqueeze a token dimension between batch and embedding dims
    random_samples = random_samples.unsqueeze(dim=1)
    #we de-embed these rows
    random_embeddings = model.unembed(random_samples)
    random_tokens = torch.argmax(random_embeddings, dim=-1)
    random_subject_str = [
        model.to_string(t) for t in random_tokens
    ]
    corrupted_facts = [
        prompt.format(s) for s in random_subject_str
    ]
    true_fact = prompt.format(subject)
    
    fact_tokens = model.to_tokens(true_fact)
    subject_mask = torch.zeros_like(fact_tokens, dtype=torch.bool, device=device)
    for value in subject_tokens[0, 1:]:
        subject_mask |= (fact_tokens == value)    
    
    return true_fact, corrupted_facts, target, subject_mask.to("cpu")
    

resample_ablation(prompt, subject, target, n_noise_samples=5)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [40]:
resample_ablation(prompt, subject, target, n_noise_samples=5)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 14.75 GiB total capacity; 13.75 GiB already allocated; 832.00 KiB free; 13.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
def pad_from_left(tokens : torch.tensor, maxlen:int, bool_tensor=False):
    pad_token = model.tokenizer.pad_token_id
    if bool_tensor: 
        pad_token = 0
    padded_tokenized_inputs = torch.zeros(tokens.shape[0], maxlen)
    
    n_pads = maxlen - tokens.shape[-1]
    padded_tokenized_inputs[:,n_pads] = pad_token
    padded_tokenized_inputs[:,n_pads:] = tokens
    return padded_tokenized_inputs.long()

def pad_to_same_length(clean_tokens, corrupted_tokens, subject_mask): 
    
    maxlen = max([clean_tokens.shape[-1], corrupted_tokens.shape[-1]])
    
    if clean_tokens.shape[-1] > corrupted_tokens.shape[-1]: 
        corrupted_tokens = pad_from_left(corrupted_tokens, maxlen)
    elif clean_tokens.shape[-1] < corrupted_tokens.shape[-1]: 
        clean_tokens = pad_from_left(clean_tokens, maxlen)
        
    subject_mask = pad_from_left(subject_mask, maxlen, bool_tensor=True)
    return clean_tokens, corrupted_tokens, subject_mask

In [30]:
def unembedding_function(residual_stack, cache, mlp=False) -> float:
    z = cache.apply_ln_to_stack(residual_stack, pos_slice=-1)
    z = z @ model.W_U
    return z

def direct_logit_attribution(clean_prompt: str,
                             corrupted_prompts: List[str],
                             subject_mask, 
                             target: str):
    clean_tokens = model.to_tokens(clean_prompt, prepend_bos=True) 
    corrupted_tokens = model.to_tokens(corrupted_prompts, prepend_bos=True)
    clean_tokens, corrupted_tokens, subject_mask = pad_to_same_length(clean_tokens, corrupted_tokens, subject_mask)
    n_tokens = clean_tokens.shape[-1]
    
    target_token = torch.tensor(model.to_single_token(f" {target}"))

    clean_tokens = clean_tokens.expand(corrupted_tokens.shape[0], -1)
    
    clean_logits, clean_cache = model.run_with_cache(clean_tokens)
    corrupted_tokens, corrupted_cache = model.run_with_cache(corrupted_tokens)
    
    clean_residual, labels = clean_cache.get_full_resid_decomposition(expand_neurons=False, pos_slice=-1, return_labels=True)
    corrupted_residual, labels = corrupted_cache.get_full_resid_decomposition(expand_neurons=False, pos_slice=-1, return_labels=True)
    
    clean_residual = unembedding_function(clean_residual, clean_cache)
    corrupted_residual = unembedding_function(corrupted_residual, corrupted_cache)

    # clean_logits = clean_residual.gather(index=target_token, dim=-1)
    # corrupted_logits = corrupted_residual.gather(index=target_token, dim=-1)

    dla = (clean_logits - corrupted_logits).mean(dim=-1)
    line(dla, x=labels, title="Difference in Logits between Alex and Tim")

    

def run_experiment(indices): 
    # indices = list(range(len(dataset)))
    # random.shuffle(indices)
    # indices = indices[:n]
    
    prompt, subject, target = sample_dataset(dataset, idx=None)
    true_fact, corrupted_facts, target, subject_mask = resample_ablation(prompt, subject, target, n_noise_samples=10)
    print(true_fact)
    result = direct_logit_attribution(
        clean_prompt=true_fact, 
        corrupted_prompts=corrupted_facts, 
        target=target, 
        subject_mask=subject_mask
    )

run_experiment(indices=None)

The Eiffel Tower is located in


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 14.75 GiB total capacity; 13.75 GiB already allocated; 832.00 KiB free; 13.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [31]:

prompts = ["When Alex and Tim went to the store, Alex gave the bag to",
 "When Alex and Tim when to the store, Tim gave the bag to"]
tokens = model.to_tokens(prompts)
print(tokens)
logits, cache = model.run_with_cache(tokens)
alex_token = model.to_single_token(" Alex")
tim_token = model.to_single_token(" Tim")
# %%
decomposed_resid, labels = cache.get_full_resid_decomposition(expand_neurons=False, pos_slice=-1, return_labels=True)
decomposed_resid = cache.apply_ln_to_stack(decomposed_resid, pos_slice=-1)
print(decomposed_resid.shape)
alex_unembed = model.W_U[:, alex_token]
tim_unembed = model.W_U[:, tim_token]
logit_diff = alex_unembed - tim_unembed
# %%
dla = einops.einsum(decomposed_resid, logit_diff, "component batch d_model, d_model -> component batch")
line(dla.T, x=labels, title="Difference in Logits between Alex and Tim")
# %%
SPACE = "·"
NEWLINE="↩"
TAB = "→"
def process_token(s):
    if isinstance(s, torch.Tensor):
        s = s.item()
    if isinstance(s, np.int64):
        s = s.item()
    if isinstance(s, int):
        s = model.to_string(s)
    s = s.replace(" ", SPACE)
    s = s.replace("\n", NEWLINE+"\n")
    s = s.replace("\t", TAB)
    return s

process_tokens = lambda l: [process_token(s) for s in l]
process_tokens_index = lambda l: [f"{process_token(s)}/{i}" for i,s in enumerate(l)]

# %%
token_list = process_tokens_index(tokens[0])

layer = 9
head = 9
imshow(cache["pattern", layer][:, head], facet_col=0, facet_labels=["Tim", "Alex"], title=f"Head {head} in Layer {layer}", x=token_list, y=token_list)
# %%
tim_decomposed_resid, labels = cache.get_full_resid_decomposition(layer=9, expand_neurons=False, pos_slice=4, return_labels=True)

print(tim_decomposed_resid.shape)

unembed_vec = model.blocks[layer].attn.W_V[head] @ model.blocks[layer].attn.W_O[head] @ (tim_unembed)

line(tim_decomposed_resid[:, 0, :] @ unembed_vec, x=labels, title="IO residual stream -> unembed via L9H9")
line(tim_decomposed_resid.norm(dim=-1).T, title="Norm of Tim Resid Stream")
line((tim_decomposed_resid[:, 0, :] @ unembed_vec)/tim_decomposed_resid.norm(dim=-1)[:, 0], x=labels, title="IO residual stream -> unembed via L9H9 divided by norm")
line(tim_decomposed_resid.norm(dim=-1).T, title="Norm of Tim Resid Stream")
# %%
imshow(cache["pattern", 0][0, :, :5, :5], facet_col=0, x=token_list[:5], y=token_list[:5])
# %%

tensor([[50256,  2215,  4422,   290,  5045,  1816,   284,   262,  3650,    11,
          4422,  2921,   262,  6131,   284],
        [50256,  2215,  4422,   290,  5045,   618,   284,   262,  3650,    11,
          5045,  2921,   262,  6131,   284]], device='cuda:0')


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 14.75 GiB total capacity; 13.75 GiB already allocated; 832.00 KiB free; 13.99 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF